<a href="https://colab.research.google.com/github/pgurazada/causal_inference/blob/master/case%20studies/hillstrom/metalearners.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from xgboost import XGBClassifier, XGBRegressor

# Data

In [2]:
data_df = pd.read_csv("/content/hillstrom_clean.csv")

In [3]:
data_df.sample(5)

,recency,history,mens,womens,newbie,visit,conversion,spend,zip_code__rural,zip_code__surburban,zip_code__urban,channel__multichannel,channel__phone,channel__web,treatment
7242,1,195.50,1,0,1,0,0,0.0,0,0,1,0,1,0,2
9530,1,271.01,1,0,0,1,0,0.0,0,0,1,0,1,0,1
53353,10,367.79,1,0,1,0,0,0.0,0,0,1,1,0,0,0
53985,9,409.40,1,0,0,0,0,0.0,0,1,0,0,1,0,1
15777,4,128.60,1,0,0,0,0,0.0,0,1,0,0,0,1,1


Historical customer attributes at your disposal include:
- Recency: Months since last purchase.
- History_Segment: Categorization of dollars spent in the past year.
- History: Actual dollar value spent in the past year.
- Mens: 1/0 indicator, 1 = customer purchased Mens merchandise in the past year.
- Womens: 1/0 indicator, 1 = customer purchased Womens merchandise in the past year.
- Zip_Code: Classifies zip code as Urban, Suburban, or Rural. - Newbie: 1/0 indicator, 1 = New customer in the past twelve months. - Channel: Describes the channels the customer purchased from in the past year.
- Treatment: Mens E-Mail, Womens E-Mail, No E-Mail

Finally, we have a series of variables describing activity in the two weeks following delivery of the e-mail campaign:
- Visit: 1/0 indicator, 1 = Customer visited website in the following two weeks.
- Conversion: 1/0 indicator, 1 = Customer purchased merchandise in the following two weeks.
- Spend: Actual dollars spent in the following two weeks.

In [4]:
data_df.visit.describe()

count    64000.000000
mean         0.146781
std          0.353890
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: visit, dtype: float64

In [5]:
data_df.conversion.describe()

count    64000.000000
mean         0.009031
std          0.094604
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: conversion, dtype: float64

# Overall Impact

In [6]:
treatment_map = {
    0: 'control',
    1: 'womens_email',
    2: 'mens_email'
}

In [7]:
# Men's emailer
(
    data_df.query("(treatment == 0 | treatment == 2)")
           .groupby('treatment')
           .agg({'visit': 'mean', 'conversion': 'mean', 'spend': 'mean'})
)

,visit,conversion,spend
treatment,,,
0,0.106167,0.005726,0.652789
2,0.182757,0.012531,1.422617


In [8]:
# Women's emailer
(
    data_df.query("(treatment == 0 | treatment == 1)")
           .groupby('treatment')
           .agg({'visit': 'mean', 'conversion': 'mean', 'spend': 'mean'})
)

,visit,conversion,spend
treatment,,,
0,0.106167,0.005726,0.652789
1,0.151400,0.008837,1.077202


# CATE

##S-Learner

In [9]:
X = data_df.drop(columns=['visit', 'conversion', 'spend'])
y_visit = data_df['visit']
y_spend = data_df['spend']

In [10]:
X_train, X_test, y_visit_train, y_visit_test = train_test_split(
    X, y_visit, test_size=0.3, random_state=42
)

In [11]:
X_train, X_test, y_spend_train, y_spend_test = train_test_split(
    X, y_spend, test_size=0.3, random_state=42
)

In [12]:
X_train.shape, y_visit_train.shape

((44800, 12), (44800,))

In [13]:
slearner_visit = XGBClassifier()
slearner_spend = XGBRegressor()

*Visits*

In [14]:
slearner_visit.fit(X_train, y_visit_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [15]:
# Calculate the difference in predictions when T=1 (womens emailer) vs T=0

slearner_te_womens = (
    slearner_visit.predict_proba(X_test.assign(**{'treatment': 1}))[:, 1] -
    slearner_visit.predict_proba(X_test.assign(**{'treatment': 0}))[:, 1]
)

In [16]:
slearner_te_womens.mean()

0.042463053

In [17]:
# Calculate the difference in predictions when T=2 (womens emailer) vs T=0

slearner_te_mens = (
    slearner_visit.predict_proba(X_test.assign(**{'treatment': 2}))[:, 1] -
    slearner_visit.predict_proba(X_test.assign(**{'treatment': 0}))[:, 1]
)

In [18]:
slearner_te_mens.mean()

0.07307953

*Spends*

In [19]:
slearner_spend.fit(X_train, y_spend_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [20]:
# Calculate the difference in predictions when T=1 (womens emailer) vs T=0

slearner_te_womens = (
    slearner_spend.predict(X_test.assign(**{'treatment': 1})) -
    slearner_spend.predict(X_test.assign(**{'treatment': 0}))
)

In [21]:
slearner_te_womens.mean()

0.36973846

In [22]:
# Calculate the difference in predictions when T=2 (womens emailer) vs T=0

slearner_te_mens = (
    slearner_spend.predict(X_test.assign(**{'treatment': 2})) -
    slearner_spend.predict(X_test.assign(**{'treatment': 0}))
)

In [23]:
slearner_te_mens.mean()

0.63133675

##T-Learner

In [24]:
train_df, test_df = train_test_split(
    data_df, test_size=0.3, random_state=42
)

In [25]:
train_df.shape, test_df.shape

((44800, 15), (19200, 15))

*Visits*

In [26]:
target = 'visit'

In [27]:
tlearner_0 = XGBClassifier()
tlearner_1 = XGBClassifier()
tlearner_2 = XGBClassifier()

In [28]:
# Split data into treated and untreated
train_0_df = train_df[train_df['treatment'] == 0]
train_1_df = train_df[train_df['treatment'] == 1]
train_2_df = train_df[train_df['treatment'] == 2]

In [29]:
# Fit the models on each sample
tlearner_0.fit(train_0_df.drop(columns=['visit', 'conversion', 'spend']), train_0_df[target])
tlearner_1.fit(train_1_df.drop(columns=['visit', 'conversion', 'spend']), train_1_df[target])
tlearner_2.fit(train_2_df.drop(columns=['visit', 'conversion', 'spend']), train_2_df[target])

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [30]:
# Calculate the difference in predictions for womens campaign
tlearner_te_womens = (
    tlearner_1.predict_proba(test_df.drop(columns=['visit', 'conversion', 'spend']))[:, 1] -
    tlearner_0.predict_proba(test_df.drop(columns=['visit', 'conversion', 'spend']))[:, 1]
)

In [31]:
tlearner_te_womens.mean()

0.044131268

In [32]:
# Calculate the difference in predictions for mens campaign
tlearner_te_mens = (
    tlearner_2.predict_proba(test_df.drop(columns=['visit', 'conversion', 'spend']))[:, 1] -
    tlearner_0.predict_proba(test_df.drop(columns=['visit', 'conversion', 'spend']))[:, 1]
)

In [33]:
tlearner_te_mens.mean()

0.074996725

*Spends*

In [34]:
target = 'spend'

In [35]:
tlearner_0 = XGBRegressor()
tlearner_1 = XGBRegressor()
tlearner_2 = XGBRegressor()

In [36]:
# Split data into treated and untreated
train_0_df = train_df[train_df['treatment'] == 0]
train_1_df = train_df[train_df['treatment'] == 1]
train_2_df = train_df[train_df['treatment'] == 2]

In [37]:
# Fit the models on each sample
tlearner_0.fit(train_0_df.drop(columns=['visit', 'conversion', 'spend']), train_0_df[target])
tlearner_1.fit(train_1_df.drop(columns=['visit', 'conversion', 'spend']), train_1_df[target])
tlearner_2.fit(train_2_df.drop(columns=['visit', 'conversion', 'spend']), train_2_df[target])

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [38]:
# Calculate the difference in predictions for womens campaign
tlearner_te_womens = (
    tlearner_1.predict(test_df.drop(columns=['visit', 'conversion', 'spend'])) -
    tlearner_0.predict(test_df.drop(columns=['visit', 'conversion', 'spend']))
)

In [39]:
tlearner_te_womens.mean()

0.539606

In [40]:
# Calculate the difference in predictions for mens campaign
tlearner_te_mens = (
    tlearner_2.predict(test_df.drop(columns=['visit', 'conversion', 'spend'])) -
    tlearner_0.predict(test_df.drop(columns=['visit', 'conversion', 'spend']))
)

In [41]:
tlearner_te_mens.mean()

0.758663

##X-Learner

*Visits*

In [42]:
target = 'visit'

In [43]:
train_df, test_df = train_test_split(
    data_df, test_size=0.3, random_state=42
)

In [44]:
train_df.shape, test_df.shape

((44800, 15), (19200, 15))

In [45]:
xlearner_0 = XGBClassifier()
xlearner_1 = XGBClassifier()
xlearner_2 = XGBClassifier()

In [46]:
# Split data into treated and untreated
train_0_df = train_df[train_df['treatment'] == 0]
train_1_df = train_df[train_df['treatment'] == 1]
train_2_df = train_df[train_df['treatment'] == 2]

In [47]:
# Fit individual models
xlearner_0.fit(train_0_df.drop(columns=['visit', 'conversion', 'spend']), train_0_df[target])
xlearner_1.fit(train_1_df.drop(columns=['visit', 'conversion', 'spend']), train_1_df[target])
xlearner_2.fit(train_2_df.drop(columns=['visit', 'conversion', 'spend']), train_2_df[target])

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

For womens campaign

In [48]:
target_columns = ['visit', 'conversion', 'spend']

In [49]:
# Calculate the difference between actual outcomes and predictions
xlearner_te_0 = xlearner_1.predict_proba(train_0_df.drop(columns=target_columns))[:, 1] - train_0_df[target]
xlearner_te_1 = train_1_df[target] - xlearner_0.predict_proba(train_1_df.drop(columns=target_columns))[:, 1]

In [50]:
xlearner_combined = XGBRegressor()

In [51]:
# Fit the combined model
xlearner_combined.fit(
  # Stack the X variables for the treated and untreated users
  pd.concat([train_0_df.drop(columns=target_columns), train_1_df.drop(columns=target_columns)]),
  # Stack the X-learner treatment effects for treated and untreated users
  pd.concat([xlearner_te_0, xlearner_te_1])
)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [52]:
xlearner_simple_te = xlearner_combined.predict(test_df.drop(columns=target_columns))

In [53]:
xlearner_simple_te.mean()

0.044232655

For mens campaign

In [54]:
target_columns = ['visit', 'conversion', 'spend']

In [55]:
# Calculate the difference between actual outcomes and predictions
xlearner_te_0 = xlearner_2.predict_proba(train_0_df.drop(columns=target_columns))[:, 1] - train_0_df[target]
xlearner_te_2 = train_2_df[target] - xlearner_0.predict_proba(train_2_df.drop(columns=target_columns))[:, 1]

In [56]:
xlearner_combined = XGBRegressor()

In [57]:
# Fit the combined model
xlearner_combined.fit(
  # Stack the X variables for the treated and untreated users
  pd.concat([train_0_df.drop(columns=target_columns), train_2_df.drop(columns=target_columns)]),
  # Stack the X-learner treatment effects for treated and untreated users
  pd.concat([xlearner_te_0, xlearner_te_2])
)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [58]:
xlearner_simple_te = xlearner_combined.predict(test_df.drop(columns=target_columns))

In [59]:
xlearner_simple_te.mean()

0.07286389

*Spends*

In [60]:
target = 'spend'

In [61]:
train_df, test_df = train_test_split(
    data_df, test_size=0.3, random_state=42
)

In [62]:
train_df.shape, test_df.shape

((44800, 15), (19200, 15))

In [63]:
xlearner_0 = XGBRegressor()
xlearner_1 = XGBRegressor()
xlearner_2 = XGBRegressor()

In [64]:
# Split data into treated and untreated
train_0_df = train_df[train_df['treatment'] == 0]
train_1_df = train_df[train_df['treatment'] == 1]
train_2_df = train_df[train_df['treatment'] == 2]

In [65]:
xlearner_0.fit(train_0_df.drop(columns=['visit', 'conversion', 'spend']), train_0_df[target])
xlearner_1.fit(train_1_df.drop(columns=['visit', 'conversion', 'spend']), train_1_df[target])
xlearner_2.fit(train_2_df.drop(columns=['visit', 'conversion', 'spend']), train_2_df[target])

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

For womens campaign

In [66]:
target_columns = ['visit', 'conversion', 'spend']

In [67]:
# Calculate the difference between actual outcomes and predictions
xlearner_te_0 = xlearner_1.predict(train_0_df.drop(columns=target_columns)) - train_0_df[target]
xlearner_te_1 = train_1_df[target] - xlearner_0.predict(train_1_df.drop(columns=target_columns))

In [68]:
xlearner_combined = XGBRegressor()

In [69]:
# Fit the combined model
xlearner_combined.fit(
  # Stack the X variables for the treated and untreated users
  pd.concat([train_0_df.drop(columns=target_columns), train_1_df.drop(columns=target_columns)]),
  # Stack the X-learner treatment effects for treated and untreated users
  pd.concat([xlearner_te_0, xlearner_te_1])
)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [70]:
xlearner_simple_te = xlearner_combined.predict(test_df.drop(columns=target_columns))

In [71]:
xlearner_simple_te.mean()

0.3512824

For mens campaign

In [72]:
target_columns = ['visit', 'conversion', 'spend']

In [73]:
# Calculate the difference between actual outcomes and predictions
xlearner_te_0 = xlearner_2.predict(train_0_df.drop(columns=target_columns)) - train_0_df[target]
xlearner_te_2 = train_2_df[target] - xlearner_0.predict(train_2_df.drop(columns=target_columns))

In [74]:
xlearner_combined = XGBRegressor()

In [75]:
# Fit the combined model
xlearner_combined.fit(
  # Stack the X variables for the treated and untreated users
  pd.concat([train_0_df.drop(columns=target_columns), train_2_df.drop(columns=target_columns)]),
  # Stack the X-learner treatment effects for treated and untreated users
  pd.concat([xlearner_te_0, xlearner_te_2])
)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [76]:
xlearner_simple_te = xlearner_combined.predict(test_df.drop(columns=target_columns))

In [77]:
xlearner_simple_te.mean()

0.9547584

Full X-learner for spends

For womens campaign.

In [78]:
# Define the new models that are not used in the simple version
xlearner_te_model_0 = XGBRegressor()
xlearner_te_model_1 = XGBRegressor()
xlearner_te_model_2 = XGBRegressor()
xlearner_propensity = XGBClassifier()

In [79]:
xlearner_te_model_0.fit(train_0_df.drop(columns=target_columns), xlearner_te_0)
xlearner_te_model_1.fit(train_1_df.drop(columns=target_columns), xlearner_te_1)
xlearner_te_model_2.fit(train_2_df.drop(columns=target_columns), xlearner_te_2)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [80]:
# Calculate predictions from the three models
xlearner_te_model_0_te = xlearner_te_model_0.predict(test_df.drop(columns=target_columns))
xlearner_te_model_1_te = xlearner_te_model_1.predict(test_df.drop(columns=target_columns))
xlearner_te_model_2_te = xlearner_te_model_2.predict(test_df.drop(columns=target_columns))

In [81]:
# Calculate the propensity scores
xlearner_propensity.fit(train_df.drop(columns=target_columns+['treatment']), train_df['treatment'])
xlearner_propensities_1 = xlearner_propensity.predict_proba(test_df.drop(columns=target_columns+['treatment']))[:, 1]
xlearner_propensities_2 = xlearner_propensity.predict_proba(test_df.drop(columns=target_columns+['treatment']))[:, 2]

In [82]:
# Calculate the treatment effects as propensity weighted average
xlearner_te_1 = xlearner_propensities_1 * xlearner_te_model_0_te + (1 - xlearner_propensities_1) * xlearner_te_model_1_te
xlearner_te_2 = xlearner_propensities_2 * xlearner_te_model_0_te + (1 - xlearner_propensities_2) * xlearner_te_model_2_te

In [83]:
xlearner_te_1.mean(), xlearner_te_2.mean()

(0.67762697, 0.8164647)